In [12]:
# Import the dependencies.
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from citipy import citipy
import json
import subprocess
import psycopg2

In [13]:
# Import the API key.
from config import client_id, client_secret, redirect_url, base_url, oauth_url

url = "https://api.procore.com/oauth/token?grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret

In [14]:
response = requests.post(url).json()
token = response.get('access_token')

In [15]:
get_company = "https://api.procore.com/rest/v1.0/companies/7930/project_regions"

data = requests.get(get_company, headers={"Authorization": "Bearer " + token}).json()

In [16]:
regions_df = pd.json_normalize(data)

regions_df

,id,name
0,243,Austin
1,244,San Antonio
2,245,Dallas
3,246,Houston
4,5383,College Station
5,11942,Louisiana


In [17]:
conn = psycopg2.connect(
   database="Master_DB", user='OdenAdmin', password= "aJDbULW45C93zCykCxAp", host='oht-rds.cbunnafql9ym.us-east-2.rds.amazonaws.com', port= '5432'
)

#Setting auto commit false
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Define SQL execution command
insert_into_table = ("""INSERT INTO "OHT_regions" ("Region", "Region Name") VALUES(%s,%s);""")

# Iterate through DF and execute commands to add data to table
for i, row in regions_df.iterrows():
   row_to_insert = (row['id'], row['name'])
   cursor.execute(insert_into_table, row_to_insert)

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()